In [ ]:
from transformers import AutoTokenizer, AutoModel, AutoConfig
from peft import PeftConfig, PeftModel
import torch
import accelerate
import bitsandbytes

In [ ]:
# Paths to your models
model_paths = ["./models/finetuned_model_0", "./models/finetuned_model_1", "./models/finetuned_model_2"]

In [ ]:
model_name = "THUDM/chatglm2-6b"
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
model = AutoModel.from_pretrained(model_name, trust_remote_code=True, load_in_8bit=True, device="cuda")

In [ ]:
model = PeftModel.from_pretrained(model, model_paths[0], adapter_name='Institute_0')
_ = model.load_adapter(model_paths[1], adapter_name='Institute_1')
_ = model.load_adapter(model_paths[2], adapter_name='Institute_2')

In [ ]:
adapters = ["Institute_0", "Institute_1", "Institute_2"]
weights = [1.0, 1.0, 1.0]
adapter_name = "merge"
density = 0.2
model.add_weighted_adapter(adapters, weights, adapter_name, combination_type="linear")

In [ ]:
model.set_adapter("merge")

In [ ]:
# Save the new model
output_model_path = "./aggregated_model"
model.save_pretrained(output_model_path)

In [ ]:
model.add_weighted_adapter(adapters, weights, "merge_ties", combination_type="ties", density=density)
output_model_path = "./aggregated_model"
model.save_pretrained(output_model_path)

##